# **Klasfikasi Suara Gabungan (Perintah + Orang)**

Pengenalan suara adalah teknologi yang memungkinkan komputer atau mesin 
untuk mengenali, memahami, dan memproses ucapan manusia sehingga dapat 
diterjemahkan menjadi perintah atau teks. 

Dalam analisis ini, saya membangun sebuah model yang mampu mengklasifikasikan suara berdasarkan **perintah (`buka`/`tutup`)** dan **siapa yang berbicara (`sahl`/`naufal`)** secara bersamaan. Model ini akan dilatih untuk mengenali empat kelas yang berbeda:
- `buka_sahl`
- `tutup_sahl`
- `buka_naufal`
- `tutup_naufal`

Dataset yang digunakan berasal dari rekaman suara pribadi. Tujuan akhirnya adalah menghasilkan satu model klasifikasi yang siap untuk digunakan dalam tahap *deployment*.

## **1. Data Understanding & Feature Extraction**

### **1.1 Install & Import Dependency**

In [ ]:
pip install librosa soundfile numpy matplotlib seaborn scikit-learn pydub ffmpeg-python joblib

In [3]:
from pydub import AudioSegment
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import scipy.stats
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
import seaborn as sns

### **1.2 Ekstraksi Fitur dari Data Suara**

Pada tahap ini, semua file audio (`.m4a` atau `.wav`) akan diproses untuk mengekstrak fitur-fitur statistik yang relevan. Target klasifikasi (`label`) akan digabungkan menjadi satu, contohnya `buka_sahl`.

In [4]:
DATA_DIR = "data_suara"
labels = ["buka_sahl", "tutup_sahl", "buka_naufal", "tutup_naufal"]

features = []
full_labels = []

def extract_stats(arr):
    return [
        np.mean(arr),
        np.std(arr),
        np.min(arr),
        np.max(arr),
        scipy.stats.skew(arr),
        scipy.stats.kurtosis(arr)
    ]

# === Loop ekstraksi fitur audio ===
for label in labels:
    folder = os.path.join(DATA_DIR, label)
    if not os.path.exists(folder):
        print(f"Folder {folder} tidak ditemukan, skip.")
        continue

    for file in os.listdir(folder):
        if file.endswith(".m4a") or file.endswith(".wav"):
            file_path = os.path.join(folder, file)

            if file.endswith(".m4a"):
                wav_path = file_path.replace(".m4a", ".wav")
                try:
                    audio = AudioSegment.from_file(file_path, format="m4a")
                    audio.export(wav_path, format="wav")
                    file_path = wav_path
                except Exception as e:
                    print(f"Gagal konversi {file_path}: {e}")
                    continue
            
            y, sr = librosa.load(file_path, sr=None)

            if np.max(np.abs(y)) > 0:
                y = y / np.max(np.abs(y))

            feat_vector = []

            # Fitur statistik dari sinyal mentah
            feat_vector.extend(extract_stats(y))

            # Fitur temporal
            zcr = librosa.feature.zero_crossing_rate(y)[0]
            feat_vector.extend(extract_stats(zcr))
            rms = librosa.feature.rms(y=y)[0]
            feat_vector.extend(extract_stats(rms))

            # Fitur spektral
            centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
            feat_vector.extend(extract_stats(centroid))
            bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
            feat_vector.extend(extract_stats(bandwidth))
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
            feat_vector.extend(extract_stats(rolloff))
            flatness = librosa.feature.spectral_flatness(y=y)[0]
            feat_vector.extend(extract_stats(flatness))
            contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
            for band in contrast:
                feat_vector.extend(extract_stats(band))
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            for bin_chroma in chroma:
                feat_vector.extend(extract_stats(bin_chroma))

            # MFCC (40 koefisien) + delta + delta-delta
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
            delta = librosa.feature.delta(mfcc)
            delta2 = librosa.feature.delta(mfcc, order=2)

            for coeff in mfcc:
                feat_vector.extend(extract_stats(coeff))
            for coeff in delta:
                feat_vector.extend(extract_stats(coeff))
            for coeff in delta2:
                feat_vector.extend(extract_stats(coeff))

            # Tambahkan ke dataset dengan satu label gabungan
            features.append(feat_vector)
            full_labels.append(label)

# === Buat DataFrame ===
df = pd.DataFrame(features)
df["label"] = full_labels

print(f"Total data: {len(df)}")
print(f"Jumlah fitur per file: {df.shape[1]-1}")
print("\nDistribusi Label:")
print(df['label'].value_counts())
print("\nData Head:")
print(df.head())

Total data: 800
Jumlah fitur per file: 876

Distribusi Label:
label
buka_sahl       200
tutup_sahl      200
buka_naufal     200
tutup_naufal    200
Name: count, dtype: int64

Data Head:
          0         1         2         3         4          5         6  \
0  0.000012  0.117857 -0.627928  1.000000  1.850656  14.627781  0.036750   
1  0.000012  0.117857 -0.627928  1.000000  1.850656  14.627781  0.036750   
2 -0.000134  0.146948 -0.617306  1.000000  1.545906   7.272887  0.026841   
3 -0.000134  0.146948 -0.617306  1.000000  1.545906   7.272887  0.026841   
4 -0.000354  0.255693 -1.000000  0.966238  0.315259   2.364644  0.029914   

          7    8         9  ...       867       868       869       870  \
0  0.023301  0.0  0.097656  ...  0.791086 -0.127478 -0.479910 -0.014375   
1  0.023301  0.0  0.097656  ...  0.791086 -0.127478 -0.479910 -0.014375   
2  0.019015  0.0  0.091797  ...  1.225600 -0.113959  0.036819  0.022988   
3  0.019015  0.0  0.091797  ...  1.225600 -0.113959  0.03

## **2. Preprocessing, Training & Validasi Model**

### **2.1 Seleksi Fitur, Reduksi Dimensi, dan Pelatihan Model**

Proses ini menggabungkan beberapa langkah:
1.  **Seleksi Fitur**: Menggunakan *Mutual Information* untuk memilih fitur-fitur yang paling informatif.
2.  **Reduksi Dimensi**: Menggunakan PCA untuk mengurangi jumlah fitur dan menangkap variasi data yang paling penting.
3.  **Pelatihan**: Melatih model *Ensemble Voting Classifier* yang terdiri dari SVM, RandomForest, dan GradientBoosting.
4.  **Validasi**: Menggunakan *Stratified K-Fold Cross-Validation* untuk mendapatkan estimasi performa model yang lebih robust.

In [ ]:
# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=["label"])
y = df["label"]

# --- 1. Seleksi Fitur dengan Mutual Information ---
# Berdasarkan analisis sebelumnya, threshold IG 0.3 untuk target 'orang' 
# (yang lebih sulit) memberikan hasil baik, kita gunakan nilai ini sebagai awal.
IG_THRESHOLD = 0.2  # Nilai ini bisa disesuaikan kembali

mi_scores = mutual_info_classif(X, y)
selected_features = X.columns[mi_scores > IG_THRESHOLD]
X_selected = X[selected_features]

print(f"Jumlah fitur setelah seleksi IG (threshold > {IG_THRESHOLD}): {X_selected.shape[1]}")

# --- 2. Reduksi Dimensi dengan PCA ---
PCA_VARIANCE = 0.95 # Mempertahankan 95% varians
pca = PCA(n_components=PCA_VARIANCE)
X_pca = pca.fit_transform(X_selected)

print(f"Jumlah komponen PCA (variance > {PCA_VARIANCE}): {X_pca.shape[1]}")

# --- 3. Definisi Model Ensemble ---
svm = SVC(kernel="rbf", C=10, gamma="scale", probability=True, random_state=42)
rf = RandomForestClassifier(n_estimators=200, random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, random_state=42)

ensemble_model = VotingClassifier(
    estimators=[("svm", svm), ("rf", rf), ("gb", gb)],
    voting="soft"
)

# --- 4. Cross-Validation ---
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(ensemble_model, X_pca, y, cv=cv, scoring="accuracy")

print(f"\nHasil Cross-Validation (5-fold):")
print(f"Akurasi per fold: {cv_scores}")
print(f"Rata-rata Akurasi CV: {np.mean(cv_scores):.4f}")
print(f"Standar Deviasi Akurasi CV: {np.std(cv_scores):.4f}")

## **3. Evaluasi & Penyimpanan Model Final**

### **3.1 Latih & Evaluasi pada Data Test**

Setelah divalidasi, model akhir dilatih pada seluruh data training dan dievaluasi performanya pada data test yang belum pernah dilihat sebelumnya.

In [ ]:
# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# Latih model final pada seluruh data training
ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)

# Evaluasi final
final_accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model Final pada Data Test: {final_accuracy:.4f}\n")

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Visualisasi confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix Model Final")
plt.show()

### **3.2 Simpan Model dan Objek Preprocessing**

Menyimpan model yang telah dilatih beserta semua komponen prapemrosesan (`pca`, `selected_features`) agar dapat digunakan kembali saat deployment untuk memproses data baru.

In [ ]:
# Latih ulang model dan PCA pada keseluruhan data untuk deployment
print("Melatih ulang model pada keseluruhan data...")
final_pca = PCA(n_components=PCA_VARIANCE)
final_X_pca = final_pca.fit_transform(X_selected) # X_selected dari sel sebelumnya

final_model = VotingClassifier(
    estimators=[("svm", svm), ("rf", rf), ("gb", gb)],
    voting="soft"
)
final_model.fit(final_X_pca, y)

# Simpan semua komponen yang diperlukan
joblib.dump(final_model, "model_klasifikasi_suara.pkl")
joblib.dump(final_pca, "pca_transform.pkl")
joblib.dump(selected_features, "selected_features_ig.pkl")
joblib.dump(list(X.columns), "semua_fitur.pkl") # Untuk referensi urutan fitur

print("\nModel dan objek preprocessing berhasil disimpan:")
print("- model_klasifikasi_suara.pkl")
print("- pca_transform.pkl")
print("- selected_features_ig.pkl")
print("- semua_fitur.pkl")